In [1]:
!pip install sumy

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 67.9 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21691 sha256=0b4a5e6e6510ee3092fafaec40e6908c081988a60829bdb2e87e63fb67b94f40
  Stored in directory: /root/.cache/pip/wheels/4d/57/58/7e3d7fedf51fe248b7fcee3df6945ae28638e22cddf01eb92b
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=61aaf4ae630168a807e6080501d480686356dec0de403bc0a01696925162556c
  Stored in directory: /root/.cache/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc74481cca4a78a9ce
Successfully built breadability docopt


In [2]:
import pandas as pd
import ast
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer

# Load dataset
df = pd.read_csv('/kaggle/input/pubmed-abstracts/pubmed_abstracts.csv')

# Parse deep learning column
deep_df = df[['deep_learning']].copy()
deep_df.dropna(inplace=True)

# Convert the stringified tuple to actual list and title
deep_df['parsed'] = deep_df['deep_learning'].apply(lambda x: ast.literal_eval(x))
deep_df[['sentences', 'title']] = pd.DataFrame(deep_df['parsed'].tolist(), index=deep_df.index)

# Combine sentences into full abstract text
deep_df['full_abstract'] = deep_df['sentences'].apply(lambda sents: " ".join(sents))

# Setup TextRank summarizer
def summarize_text(text, sentence_count=2):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = TextRankSummarizer()
    summary = summarizer(parser.document, sentence_count)
    return " ".join([str(sentence) for sentence in summary])

# Generate summaries for top 100 articles (adjust as needed)
deep_df['summary'] = deep_df['full_abstract'].apply(lambda x: summarize_text(x, sentence_count=2))

# Preview results
deep_df[['title', 'summary']].head()

,title,summary
0,Constrained Magnetic Resonance Spectroscopic I...,A regularization formulation is proposed to ef...
1,Lightweight Pyramid Networks for Image Deraining.,"In particular, we find that by introducing the..."
2,Thorax-Net: An Attention Regularized Deep Neur...,The attention branch exploits the correlation ...
3,Correction to: Classification of Carotid Arter...,The original article unfortunately contained a...
4,Artificial Intelligence in Drug Treatment.,The most common applications of artificial int...


In [3]:
deep_df[['title', 'summary']].to_csv("summarized_articles.csv", index=False)